# Get whole instagram network adjlists

In [ ]:
#!pip install selenium==3.141.0

In [2]:
#!pip install pyvis

  Created wheel for pyvis: filename=pyvis-0.2.1-py3-none-any.whl size=23670 sha256=b5b67250c9f99ad24a32f29a46e29d7f1a55c02b475942e3b2603d09f9f14845
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\78\bb\02\40c79afcad1a632f7905a585ed3c5881737550400ff74ab6b5
Successfully built pyvis


In [3]:
import time
from functools import reduce
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [4]:
#READ CREDENTIALS AND USER PROFILE
config=open("instagram_scrapper_conf.txt","r")
lines=config.readlines()
USERNAME=lines[0].split("USER_EMAIL=")[1].replace("\n","")
PASSWORD=lines[1].split("USER_PASSWORD=")[1].replace("\n","")
PATH=lines[2].split("PATH=")[1].replace("\n","")
config.close()

In [5]:
driver = webdriver.Chrome(executable_path = PATH)
driver.maximize_window()
url="https://www.instagram.com"
driver.get(url)
time.sleep(3)
#refresh_if_not_loaded(driver, "//input[contains(@name,'username')]")
driver.find_element_by_xpath(
        "//input[contains(@name,'username')]").send_keys(USERNAME) # Sending Email
driver.find_element_by_xpath(
        "//input[@name='password']").send_keys(PASSWORD) # Sending Password
driver.find_element_by_xpath("//button[@class='sqdOP  L3NKy   y3zKF     ']").click() # Clicking Log In
time.sleep(5)
driver.find_element_by_xpath("//button[normalize-space()='Ahora no']").click()
time.sleep(5)
driver.find_element_by_xpath("//button[normalize-space()='Ahora no']").click()

In [13]:
def get_count():
    """Get number of both followers and following"""
    
    follower_count=driver.find_element_by_xpath(
    "(//div[@class='_aacl _aacp _aacu _aacx _aad6 _aade'])[2]")
    following_count=driver.find_element_by_xpath(
    "(//div[@class='_aacl _aacp _aacu _aacx _aad6 _aade'])[3]")
    followers=int(follower_count.text.replace(",", "").split(" ")[0])
    following=int(following_count.text.replace(",", "").split(" ")[0])
    return followers, following

In [14]:
def get_people_list(XPATH):
    """"""
    scrollable=driver.find_element_by_xpath(
                "//div[contains(@class,'_aano')]")
    list_of_people = []
    while True:
        scroll_size=driver.execute_script("return document.querySelector('._aano').scrollHeight")
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable)
        time.sleep(2)
        new_scroll_size=driver.execute_script("return document.querySelector('._aano').scrollHeight")
        if new_scroll_size==scroll_size:
            break 
    list_of_people=driver.find_elements_by_xpath(XPATH)
    
    final_list=[]
    for ele in list_of_people:
        final_list.append(str(ele.text))
        
    return final_list

In [15]:
def get_people(profile):
    """"""
    driver.get("https://www.instagram.com/{}/".format(profile))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "(//div[@class='_aacl _aacp _aacu _aacx _aad6 _aade'])[2]")))
    followers, following = get_count()
    #print(followers,following)
    
    # Get followers
    driver.get("https://www.instagram.com/{}/followers".format(profile))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'_aano')]")))
    list_of_followers = get_people_list("//span[contains(@class,'_aacl _aaco _aacw _aacx _aad7 _aade')]")
    
    # Get following
    driver.get("https://www.instagram.com/{}/following/".format(profile))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'_aano')]")))
    list_of_following = get_people_list("//span[contains(@class,'_aacl _aaco _aacw _aacx _aad7 _aade')]")
    
    return list_of_followers, list_of_following

In [16]:
def get_intersection(list_of_followers, list_of_following):
    A=set(list_of_followers)
    B=set(list_of_following)
    return list(A.intersection(B))

In [17]:
def generate_file(profile, intersection):
    f = open("adjlists/" + profile+".adjlist", "w")
    for ele in intersection:
        f.write(profile + " " + ele + "\n")
    f.close()

In [18]:
def generate_network(profile):
    """ """
    list_of_followers, list_of_following = get_people(profile)
    intersection=get_intersection(list_of_followers, list_of_following)
    generate_file(profile,intersection)
    G = nx.read_adjlist("adjlists/" + "{}.adjlist".format(profile))
    
    for ele in intersection:
        try:
            list_of_followers, list_of_following = get_people(ele)
            new_intersection=get_intersection(list_of_followers, list_of_following)
            generate_file(ele, new_intersection)
            Ga = nx.read_adjlist("adjlists/" + "{}.adjlist".format(ele))
            G.add_edges_from(Ga.edges)
        except:
            continue
    return G

In [19]:
# MAIN
if __name__ == '__main__':
    Graph = generate_network("gabosalazarb")
    nx.write_adjlist(Graph, "whole_network.adjlist")
    nt = Network('900px', '1500px')
    nt.from_nx(Graph)
    nt.show('whole_network.html')
    driver.close()